# Working with the FERC Form 1 Extract / Transform
This notebook steps through PUDL's extract and transform steps for FERC Form 1 to make it easier to test and add new years of data, or new tables from the various spreadsheets that haven't been integrated yet.

This notebook deviates from other devtool debug notebooks in that it doesn't make use of the most recently created dagster asset values. Instead, the extraction and transforms steps are rerun within the notebook so we can inspect the outputs of lower level transform functions that don't have their own assets like `process_xbrl` and `transform_start`.

**Make sure you've created the raw FERC databases using one of the `ferc_to_sqlite` jobs!**

## Setup

In [1]:
%load_ext autoreload
%autoreload 3
import pudl
import logging
import sys
from pathlib import Path
import pandas as pd
pd.options.display.max_columns = None

pudl_settings is being deprecated in favor of environment variables variables PUDL_OUTPUT and PUDL_INPUT. For more info see: https://catalystcoop-pudl.readthedocs.io/en/dev/dev/dev_setup.html
pudl_settings is being deprecated in favor of environment variables PUDL_OUTPUT and PUDL_INPUT. For more info see: https://catalystcoop-pudl.readthedocs.io/en/dev/dev/dev_setup.html
sqlite and parquet directories are no longer being used. Make sure there is a single directory named 'output' at the root of your workspace. For more info see: https://catalystcoop-pudl.readthedocs.io/en/dev/dev/dev_setup.html
pudl_settings is being deprecated in favor of environment variables variables PUDL_OUTPUT and PUDL_INPUT. For more info see: https://catalystcoop-pudl.readthedocs.io/en/dev/dev/dev_setup.html
pudl_settings is being deprecated in favor of environment variables PUDL_OUTPUT and PUDL_INPUT. For more info see: https://catalystcoop-pudl.readthedocs.io/en/dev/dev/dev_setup.html
sqlite and parquet direct

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

## Extract DBF and XBRL Data:

In [3]:
from dagster import build_init_resource_context
from pudl.resources import dataset_settings

years = [2020, 2021] # add desired years here
configured_dataset_settings = {'ferc1': {'years': years}}

dataset_init_context = build_init_resource_context(config=configured_dataset_settings)
configured_dataset_settings = dataset_settings(dataset_init_context)

In [4]:
from pudl.extract.ferc1 import extract_dbf, extract_xbrl

ferc1_dbf_raw_dfs = extract_dbf(configured_dataset_settings)
ferc1_xbrl_raw_dfs = extract_xbrl(configured_dataset_settings)

2023-03-17 17:02:10 [ WARNING] catalystcoop.pudl.io_managers:166 steam_electric_generating_plant_statistics_large_plants_fuel_statistics_402_instant not found in database metadata. Dtypes of returned DataFrame might be incorrect.


steam_electric_generating_plant_statistics_large_plants_fuel_statistics_402_instant not found in database metadata. Dtypes of returned DataFrame might be incorrect.


2023-03-17 17:02:10 [ WARNING] catalystcoop.pudl.io_managers:166 purchased_power_326_instant not found in database metadata. Dtypes of returned DataFrame might be incorrect.


purchased_power_326_instant not found in database metadata. Dtypes of returned DataFrame might be incorrect.


2023-03-17 17:02:10 [ WARNING] catalystcoop.pudl.io_managers:166 electric_energy_account_401a_instant not found in database metadata. Dtypes of returned DataFrame might be incorrect.


...

sales_of_electricity_by_rate_schedules_account_448_interdepartmental_sales_304_instant not found in database metadata. Dtypes of returned DataFrame might be incorrect.


2023-03-17 17:02:11 [ WARNING] catalystcoop.pudl.io_managers:166 sales_of_electricity_by_rate_schedules_account_4491_provision_for_rate_refunds_304_instant not found in database metadata. Dtypes of returned DataFrame might be incorrect.


sales_of_electricity_by_rate_schedules_account_4491_provision_for_rate_refunds_304_instant not found in database metadata. Dtypes of returned DataFrame might be incorrect.


2023-03-17 17:02:11 [ WARNING] catalystcoop.pudl.io_managers:166 sales_of_electricity_by_rate_schedules_account_totals_304_instant not found in database metadata. Dtypes of returned DataFrame might be incorrect.


sales_of_electricity_by_rate_schedules_account_totals_304_instant not found in database metadata. Dtypes of returned DataFrame might be incorrect.


In [5]:
ferc1_xbrl_raw_dfs["fuel_ferc1"]["duration"].report_year

0       2021
1       2021
2       2021
3       2021
4       2021
        ... 
1313    2021
1314    2021
1315    2021
1316    2021
1317    2021
Name: report_year, Length: 1318, dtype: int64

In [6]:
from pudl.extract.ferc1 import xbrl_metadata_json
from dagster import build_op_context

context = build_op_context()
xbrl_metadata_json_dict = xbrl_metadata_json(context)

## Transform FERC 1 Tables:

### Build Transformers

In [7]:
# Get table class information
import inspect
from pudl.transform.ferc1 import *
from pudl.transform.params import *

def get_table_classes(module):
    classes = [member[1] for member in inspect.getmembers(module, inspect.isclass)]
    table_classes = [x for x in classes if x.__name__.endswith("Ferc1TableTransformer")]
    return [x for x in table_classes if x.__name__ != "AbstractFerc1TableTransformer"]

classes = get_table_classes(pudl.transform.ferc1)
table_id_dict = {clas.table_id.value: clas for clas in classes}

# Loop over selected tables to build the transformers
transformers = {}
for table in TABLE_NAME_MAP_FERC1.keys():
    # this table is in the name map but doesn't have a transform class
    if table == "retained_earnings_appropriations_ferc1":
        continue
    transformers[table] = (
        table_id_dict[table](
            xbrl_metadata_json=xbrl_metadata_json_dict[table],
            cache_dfs=True,
            clear_cached_dfs=False
        )
    )

2023-03-17 17:02:11 [    INFO] catalystcoop.pudl.transform.ferc1:993 fuel_ferc1: Processing XBRL metadata.


fuel_ferc1: Processing XBRL metadata.


2023-03-17 17:02:11 [    INFO] catalystcoop.pudl.transform.ferc1:993 plants_steam_ferc1: Processing XBRL metadata.


plants_steam_ferc1: Processing XBRL metadata.


2023-03-17 17:02:11 [    INFO] catalystcoop.pudl.transform.ferc1:993 plants_small_ferc1: Processing XBRL metadata.


...

cash_flow_ferc1: Processing XBRL metadata.


2023-03-17 17:02:12 [    INFO] catalystcoop.pudl.transform.ferc1:993 electricity_sales_by_rate_schedule_ferc1: Processing XBRL metadata.


electricity_sales_by_rate_schedule_ferc1: Processing XBRL metadata.


2023-03-17 17:02:12 [    INFO] catalystcoop.pudl.transform.ferc1:993 other_regulatory_liabilities_ferc1: Processing XBRL metadata.


other_regulatory_liabilities_ferc1: Processing XBRL metadata.


### Transform Individual Tables

In [8]:
from pprint import pprint

# Pick one table to transform
pprint(list(transformers.keys()))

['fuel_ferc1',
 'plants_steam_ferc1',
 'plants_small_ferc1',
 'plants_hydro_ferc1',
 'plants_pumped_storage_ferc1',
 'plant_in_service_ferc1',
 'purchased_power_ferc1',
 'electric_energy_sources_ferc1',
 'electric_energy_dispositions_ferc1',
 'utility_plant_summary_ferc1',
 'transmission_statistics_ferc1',
 'electric_operating_expenses_ferc1',
 'balance_sheet_liabilities_ferc1',
 'balance_sheet_assets_ferc1',
 'income_statement_ferc1',
 'retained_earnings_ferc1',
 'depreciation_amortization_summary_ferc1',
 'electric_plant_depreciation_changes_ferc1',
 'electric_plant_depreciation_functional_ferc1',
 'electric_operating_revenues_ferc1',
 'cash_flow_ferc1',
 'electricity_sales_by_rate_schedule_ferc1',
 'other_regulatory_liabilities_ferc1']


In [9]:
table_name = "other_regulatory_liabilities_ferc1"
TRANSFORMER = transformers[table_name] # add a table here

#### Test each step of the transform process:

In [10]:
xbrl = TRANSFORMER.process_xbrl(
    raw_xbrl_instant=ferc1_xbrl_raw_dfs[TRANSFORMER.table_id.value]["instant"],
    raw_xbrl_duration=ferc1_xbrl_raw_dfs[TRANSFORMER.table_id.value]["duration"]
)

2023-03-17 17:02:12 [    INFO] catalystcoop.pudl.transform.ferc1:1100 other_regulatory_liabilities_ferc1: Processing XBRL data pre-concatenation.


other_regulatory_liabilities_ferc1: Processing XBRL data pre-concatenation.


2023-03-17 17:02:12 [    INFO] catalystcoop.pudl.transform.classes:1210 other_regulatory_liabilities_ferc1: Attempting to rename 0 columns.


other_regulatory_liabilities_ferc1: Attempting to rename 0 columns.


2023-03-17 17:02:12 [    INFO] catalystcoop.pudl.transform.ferc1:1154 other_regulatory_liabilities_ferc1: Unstacking balances to the report years.


...

other_regulatory_liabilities_ferc1: Both XBRL instant & duration tables found.


2023-03-17 17:02:12 [    INFO] catalystcoop.pudl.transform.ferc1:1304 other_regulatory_liabilities_ferc1: Combining XBRL instant & duration tables using RIGHT-MERGE.


other_regulatory_liabilities_ferc1: Combining XBRL instant & duration tables using RIGHT-MERGE.


2023-03-17 17:02:12 [    INFO] catalystcoop.pudl.transform.classes:1210 other_regulatory_liabilities_ferc1: Attempting to rename 9 columns.


other_regulatory_liabilities_ferc1: Attempting to rename 9 columns.


In [11]:
dbf = TRANSFORMER.process_dbf(
    raw_dbf=ferc1_dbf_raw_dfs[TRANSFORMER.table_id.value]
)

2023-03-17 17:02:12 [    INFO] catalystcoop.pudl.transform.ferc1:1079 other_regulatory_liabilities_ferc1: Processing DBF data pre-concatenation.


other_regulatory_liabilities_ferc1: Processing DBF data pre-concatenation.


2023-03-17 17:02:12 [    INFO] catalystcoop.pudl.transform.ferc1:1142 other_regulatory_liabilities_ferc1: After selection only annual records, we have 26.7% of the original table.


other_regulatory_liabilities_ferc1: After selection only annual records, we have 26.7% of the original table.


2023-03-17 17:02:12 [    INFO] catalystcoop.pudl.transform.classes:1210 other_regulatory_liabilities_ferc1: Attempting to rename 13 columns.


other_regulatory_liabilities_ferc1: Attempting to rename 13 columns.


In [12]:
start = TRANSFORMER.transform_start(
    raw_dbf=ferc1_dbf_raw_dfs[TRANSFORMER.table_id.value],
    raw_xbrl_instant=ferc1_xbrl_raw_dfs[TRANSFORMER.table_id.value]["instant"],
    raw_xbrl_duration=ferc1_xbrl_raw_dfs[TRANSFORMER.table_id.value]["duration"]
)

2023-03-17 17:02:12 [    INFO] catalystcoop.pudl.transform.ferc1:1079 other_regulatory_liabilities_ferc1: Processing DBF data pre-concatenation.


other_regulatory_liabilities_ferc1: Processing DBF data pre-concatenation.


2023-03-17 17:02:12 [    INFO] catalystcoop.pudl.transform.ferc1:1142 other_regulatory_liabilities_ferc1: After selection only annual records, we have 26.7% of the original table.


other_regulatory_liabilities_ferc1: After selection only annual records, we have 26.7% of the original table.


2023-03-17 17:02:12 [    INFO] catalystcoop.pudl.transform.classes:1210 other_regulatory_liabilities_ferc1: Attempting to rename 13 columns.


...

other_regulatory_liabilities_ferc1: Combining XBRL instant & duration tables using RIGHT-MERGE.


2023-03-17 17:02:12 [    INFO] catalystcoop.pudl.transform.classes:1210 other_regulatory_liabilities_ferc1: Attempting to rename 9 columns.


other_regulatory_liabilities_ferc1: Attempting to rename 9 columns.


2023-03-17 17:02:12 [    INFO] catalystcoop.pudl.transform.ferc1:916 other_regulatory_liabilities_ferc1: Concatenating DBF + XBRL dataframes.


other_regulatory_liabilities_ferc1: Concatenating DBF + XBRL dataframes.


In [13]:
main = TRANSFORMER.transform_main(
    start
)

2023-03-17 17:02:13 [    INFO] catalystcoop.pudl.transform.classes:1336 other_regulatory_liabilities_ferc1: Spot fixing missing values.


other_regulatory_liabilities_ferc1: Spot fixing missing values.


2023-03-17 17:02:13 [    INFO] catalystcoop.pudl.transform.classes:1234 other_regulatory_liabilities_ferc1: Normalizing freeform string columns.


other_regulatory_liabilities_ferc1: Normalizing freeform string columns.


2023-03-17 17:02:13 [    INFO] catalystcoop.pudl.transform.classes:1258 other_regulatory_liabilities_ferc1: Categorizing string columns using a controlled vocabulary.


...

other_regulatory_liabilities_ferc1: Replacing specified values with NA.


2023-03-17 17:02:13 [    INFO] catalystcoop.pudl.transform.classes:1312 other_regulatory_liabilities_ferc1: Dropping remaining invalid rows.


other_regulatory_liabilities_ferc1: Dropping remaining invalid rows.


2023-03-17 17:02:13 [    INFO] catalystcoop.pudl.transform.classes:823 29.8% of records (2113 rows) contain only {0, nan, '', <NA>} values in required columns. Dropped these 💩💩💩 records.


29.8% of records (2113 rows) contain only {0, nan, '', <NA>} values in required columns. Dropped these 💩💩💩 records.


In [14]:
end = TRANSFORMER.transform_end(
    main
)

2023-03-17 17:02:13 [    INFO] catalystcoop.pudl.transform.classes:1343 other_regulatory_liabilities_ferc1: Enforcing database schema on dataframe.


other_regulatory_liabilities_ferc1: Enforcing database schema on dataframe.


#### Test all steps together

In [15]:
full = TRANSFORMER.transform(
    raw_dbf=ferc1_dbf_raw_dfs[TRANSFORMER.table_id.value],
    raw_xbrl_instant=ferc1_xbrl_raw_dfs[TRANSFORMER.table_id.value]["instant"],
    raw_xbrl_duration=ferc1_xbrl_raw_dfs[TRANSFORMER.table_id.value]["duration"]
)

2023-03-17 17:02:13 [    INFO] catalystcoop.pudl.transform.ferc1:1079 other_regulatory_liabilities_ferc1: Processing DBF data pre-concatenation.


other_regulatory_liabilities_ferc1: Processing DBF data pre-concatenation.


2023-03-17 17:02:13 [    INFO] catalystcoop.pudl.transform.ferc1:1142 other_regulatory_liabilities_ferc1: After selection only annual records, we have 26.7% of the original table.


other_regulatory_liabilities_ferc1: After selection only annual records, we have 26.7% of the original table.


2023-03-17 17:02:13 [    INFO] catalystcoop.pudl.transform.classes:1210 other_regulatory_liabilities_ferc1: Attempting to rename 13 columns.


...

other_regulatory_liabilities_ferc1: Dropping remaining invalid rows.


2023-03-17 17:02:13 [    INFO] catalystcoop.pudl.transform.classes:823 29.8% of records (2113 rows) contain only {0, nan, '', <NA>} values in required columns. Dropped these 💩💩💩 records.


29.8% of records (2113 rows) contain only {0, nan, '', <NA>} values in required columns. Dropped these 💩💩💩 records.


2023-03-17 17:02:13 [    INFO] catalystcoop.pudl.transform.classes:1343 other_regulatory_liabilities_ferc1: Enforcing database schema on dataframe.


other_regulatory_liabilities_ferc1: Enforcing database schema on dataframe.


### Transform All Tables

In [16]:
transformed_tables = {}
for table_name, transformer in transformers.items():
    if table_name == "plants_steam_ferc1":
        # plants_steam_ferc1 is a special case. It depends on the transformed fuel_ferc1 table.
        continue
    transformed_tables[transformer.table_id.value] = transformer.transform(
        raw_dbf=ferc1_dbf_raw_dfs[transformer.table_id.value],
        raw_xbrl_instant=ferc1_xbrl_raw_dfs[transformer.table_id.value]["instant"],
        raw_xbrl_duration=ferc1_xbrl_raw_dfs[transformer.table_id.value]["duration"]
    )

2023-03-17 17:02:13 [    INFO] catalystcoop.pudl.transform.ferc1:1079 fuel_ferc1: Processing DBF data pre-concatenation.


fuel_ferc1: Processing DBF data pre-concatenation.


2023-03-17 17:02:13 [    INFO] catalystcoop.pudl.transform.classes:1210 fuel_ferc1: Attempting to rename 17 columns.


fuel_ferc1: Attempting to rename 17 columns.


2023-03-17 17:02:13 [    INFO] catalystcoop.pudl.transform.classes:1283 fuel_ferc1: Converting units and renaming columns accordingly.


...

other_regulatory_liabilities_ferc1: Dropping remaining invalid rows.


2023-03-17 17:02:28 [    INFO] catalystcoop.pudl.transform.classes:823 29.8% of records (2113 rows) contain only {0, nan, '', <NA>} values in required columns. Dropped these 💩💩💩 records.


29.8% of records (2113 rows) contain only {0, nan, '', <NA>} values in required columns. Dropped these 💩💩💩 records.


2023-03-17 17:02:28 [    INFO] catalystcoop.pudl.transform.classes:1343 other_regulatory_liabilities_ferc1: Enforcing database schema on dataframe.


other_regulatory_liabilities_ferc1: Enforcing database schema on dataframe.


In [17]:
# Handle special case for "plants_steam_ferc1"
transformer = transformers["plants_steam_ferc1"]
transformed_tables[transformer.table_id.value] = transformer.transform(
    raw_dbf=ferc1_dbf_raw_dfs[transformer.table_id.value],
    raw_xbrl_instant=ferc1_xbrl_raw_dfs[transformer.table_id.value]["instant"],
    raw_xbrl_duration=ferc1_xbrl_raw_dfs[transformer.table_id.value]["duration"],
    transformed_fuel=transformed_tables["fuel_ferc1"]
)

2023-03-17 17:02:28 [    INFO] catalystcoop.pudl.transform.ferc1:1079 plants_steam_ferc1: Processing DBF data pre-concatenation.


plants_steam_ferc1: Processing DBF data pre-concatenation.


2023-03-17 17:02:28 [    INFO] catalystcoop.pudl.transform.classes:1210 plants_steam_ferc1: Attempting to rename 43 columns.


plants_steam_ferc1: Attempting to rename 43 columns.


2023-03-17 17:02:28 [    INFO] catalystcoop.pudl.transform.ferc1:1100 plants_steam_ferc1: Processing XBRL data pre-concatenation.


...

Successfully Identified 988 multi-year plant entities.


2023-03-17 17:02:36 [   ERROR] catalystcoop.pudl.analysis.classify_plants_ferc1:594 Found report_year=2020 2 times in plant_id_ferc1=557


Found report_year=2020 2 times in plant_id_ferc1=557


2023-03-17 17:02:36 [    INFO] catalystcoop.pudl.transform.classes:1343 plants_steam_ferc1: Enforcing database schema on dataframe.


plants_steam_ferc1: Enforcing database schema on dataframe.
